## Source: https://colab.research.google.com/drive/1KL2N-DrHmn1osEINduRJzq78jaJERFzr


In [ ]:
%%capture
!pip install wikidataintegrator
!pip install backoff
!pip install RDF

In [1]:
from wikidataintegrator import wdi_core, wdi_login
import os
from rdflib import Graph, URIRef
import pandas as pd

# This code is to Login to Wikibase. The pattern of api is the URL of the wikibase + /w/api.php
# In this example that is https://diseases.semscape.org/w/api.php


## The following section is to download the orginal Disease ontology
print("\nDownloading Mondo ...")
url = "http://purl.obolibrary.org/obo/mondo.owl"

print("\nParsing Mondo in rdflib Graph")
doGraph = Graph()
doGraph.parse(url, format="xml")



Parsing Mondo in rdflib Graph


<Graph identifier=Ncd9a88e120c64ada852987e5b0dabc58 (<class 'rdflib.graph.Graph'>)>

In [3]:
import RDF
parser=RDF.Parser(name="ntriples")
model=RDF.Model()
stream=parser.parse_into_model(model,url)
for triple in model:
    print(triple.subject, triple.predicate, triple.object)

ModuleNotFoundError: No module named 'RDF'

In [4]:
import pprint
df_mondoNative = pd.DataFrame(columns=["mondo_uri", "mondoid", "label", "exactMatches", "aliases"])

qres = doGraph.query(
    """
       PREFIX obo: <http://www.geneontology.org/formats/oboInOwl#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

       SELECT DISTINCT ?mondo_uri ?mondoid ?label (GROUP_CONCAT(?exactMatch;separator="|") as ?exactMatches) (GROUP_CONCAT(?exactsynonym;separator="|") as ?exact_synonyms)
       WHERE {
          ?mondo_uri obo:id ?mondoid ;
                  rdfs:label ?label .
         OPTIONAL {?mondo_uri <http://www.w3.org/2004/02/skos/core#exactMatch> ?exactMatch .}
         OPTIONAL {?mondo_uri oboInOwl:hasExactSynonym ?exactsynonym}
       }
       GROUP BY ?mondo_uri """)

for row in qres:
    df_mondoNative = df_mondoNative.append({
     "mondo_uri": str(row[0]),
     "mondoid": str(row[1]),
     "label":  str(row[2]),
     "exactMatches": str(row[3]),
     "aliases": str(row[4])
      }, ignore_index=True)

df_mondoNative


,mondo_uri,mondoid,label,exactMatches,aliases
0,http://purl.obolibrary.org/obo/MONDO_0014972,MONDO:0014972,"chromosome 19q13.11 deletion syndrome, proximal",http://identifiers.org/omim/617219|http://link...,
1,http://purl.obolibrary.org/obo/MONDO_0017899,MONDO:0017899,autosomal dominant mendelian susceptibility to...,http://linkedlifedata.com/resource/umls/id/CN2...,autosomal dominant MSMD due to a partial defic...
2,http://purl.obolibrary.org/obo/HP_0012210,HP:0012210,Abnormal renal morphology,,Kidney structure issue|Structural kidney abnor...
3,http://purl.obolibrary.org/obo/MONDO_0019888,MONDO:0019888,distal trisomy 20q,http://identifiers.org/snomedct/764500002|http...,telomeric duplication 20q|distal duplication 2...
4,http://purl.obolibrary.org/obo/MONDO_0009339,MONDO:0009339,congenital bile acid synthesis defect 2,http://identifiers.org/mesh/C535443|http://ide...,"CBAS2|bile acid synthesis defect, congenital, ..."
...,...,...,...,...,...
26753,http://purl.obolibrary.org/obo/MONDO_0002144,MONDO:0002144,obsolete hyperuricemia (disease),http://purl.obolibrary.org/obo/DOID_1920|http:...,hyperuricemia|uricacidemia|blood urate raized|...
26754,http://purl.obolibrary.org/obo/MONDO_0021680,MONDO:0021680,streptococcal infection,http://identifiers.org/snomedct/85769006|http:...,Streptococcus caused disease or disorder|Strep...
26755,http://purl.obolibrary.org/obo/NCBITaxon_28221,NCBITaxon:28221,Deltaproteobacteria,,
26756,http://purl.obolibrary.org/obo/MONDO_0010030,MONDO:0010030,Sjogren syndrome,http://purl.obolibrary.org/obo/DOID_12894|http...,primary Sjögren-Gougerot syndrome|primary Sjog...


In [ ]:
from wikidataintegrator.wdi_helpers import try_write
from getpass import getpass

wikibase = "https://diseases.semscape.org/w/api.php"
WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=wikibase)

query = """
  PREFIX wbt: <http://diseases.semscape.org/prop/direct/>
  SELECT * WHERE {?item wbt:P5 ?exactMatch}
"""
existing_disease = dict()
results =  wdi_core.WDItemEngine.execute_sparql_query(query=query, endpoint="https://diseases.semscape.org/query/sparql")

for result in results["results"]["bindings"]:
    existing_disease[result["exactMatch"]["value"]] = result["item"]["value"]

for index, row in df_mondoNative.iterrows():
  data = []
  mondo_reference = createMONDOReference(row["mondoid"])
  # Mondo ID
  data.append(wdi_core.WDExternalID(row["mondoid"], prop_nr="P9", references=[copy.deepcopy(mondo_reference)]))
  data.append(wdi_core.WDUrl(row["mondo_uri"], prop_nr="P5", references=[copy.deepcopy(mondo_reference)]))

  # exact matches
  for skosExactMatch in list(set(row["exactMatches"].split("|"))):
      if skosExactMatch != "":
          data.append(wdi_core.WDUrl(skosExactMatch, prop_nr="P5", references=[copy.deepcopy(mondo_reference)]))
          # Disease Ontology ID (P4)
          # http://purl.obolibrary.org/obo/DOID_xxxxx
          if "http://purl.obolibrary.org/obo/DOID" in skosExactMatch:
              doid = skosExactMatch.replace("http://purl.obolibrary.org/obo/DOID_", "DOID:")
              data.append(wdi_core.WDExternalID(doid, prop_nr="P4", references=[copy.deepcopy(mondo_reference)]))

          # MeSH descriptor ID (P8)
          # http://identifiers.org/mesh/
          if "http://identifiers.org/mesh/" in skosExactMatch:
              mesh = skosExactMatch.replace("http://identifiers.org/mesh/", "")
              data.append(wdi_core.WDExternalID(mesh, prop_nr="P8", references=[copy.deepcopy(mondo_reference)]))

          # UMLS CUI (P10)
          # http://linkedlifedata.com/resource/umls/id/
          if "http://linkedlifedata.com/resource/umls/id/" in skosExactMatch:
              umls = skosExactMatch.replace("http://linkedlifedata.com/resource/umls/id/", "")
              data.append(wdi_core.WDExternalID(umls, prop_nr="P10", references=[copy.deepcopy(mondo_reference)]))

          # Orphanet ID (P11)
          # http://www.orpha.net/ORDO/Orphanet_
          if "http://www.orpha.net/ORDO/Orphanet_" in skosExactMatch:
              ordo = skosExactMatch.replace("http://www.orpha.net/ORDO/", "")
              data.append(wdi_core.WDExternalID(ordo, prop_nr="P11", references=[copy.deepcopy(mondo_reference)]))

          # SNOMED CT identifier (P12)
          # http://identifiers.org/snomedct/191347008
          if "http://identifiers.org/snomedct/" in skosExactMatch:
              snomed = skosExactMatch.replace("http://identifiers.org/snomedct/", "")
              data.append(wdi_core.WDExternalID(snomed, prop_nr="P12", references=[copy.deepcopy(mondo_reference)]))

          # NCI Thesaurus ID (P13)
          # http://purl.obolibrary.org/obo/NCIT_
          if "http://purl.obolibrary.org/obo/NCIT_" in skosExactMatch:
              ncit = skosExactMatch.replace("http://purl.obolibrary.org/obo/NCIT_", "NCIT:")
              data.append(wdi_core.WDString(ncit, prop_nr="P13", references=[copy.deepcopy(mondo_reference)]))

          # OMIM (P14)
          # http://identifiers.org/omim/
          if "http://identifiers.org/omim/" in skosExactMatch:
              omim = skosExactMatch.replace("http://identifiers.org/omim/", "")
              data.append(wdi_core.WDExternalID(omim, prop_nr="P14", references=[copy.deepcopy(mondo_reference)]))

  qid = None
  for exactMatch in row["exactMatches"].split("|"):
    if exactMatch in existing_disease.keys():
      qid = existing_disease[exactMatch].replace("http://diseases.semscape.org/entity/", "")
    continue
  if not qid:
    wb_mondo_item = wdi_core.WDItemEngine(data=data, mediawiki_api_url=wikibase, sparql_endpoint_url="https://diseases.semscape.org/query/sparql", keep_good_ref_statements=True)
  else:
    wb_mondo_item = wdi_core.WDItemEngine(wd_item_id=qid, data=data, mediawiki_api_url=wikibase, sparql_endpoint_url="https://diseases.semscape.org/query/sparql", keep_good_ref_statements=True)
  
  if wb_mondo_item.get_label() == "":
    wb_mondo_item.set_label(row["label"], lang="en")
  if wb_mondo_item.get_description() == "":
    wb_mondo_item.set_description("human disease", lang="en")
  if row["aliases"] != "":
    print(row["aliases"])
    aliases = wb_mondo_item.get_aliases()
    for new_alias in row["aliases"].split("|"):
       aliases.append(new_alias)
    wb_mondo_item.set_aliases(aliases, lang="en")
  #pprint.pprint(wb_mondo_item.get_wd_json_representation())
  print(wb_mondo_item.wd_item_id)
  try_write(wb_mondo_item, record_id=row["mondoid"], record_prop="P4", edit_summary="Updated a MondoID", login=login)
  sys.exit()




In [ ]:
query = """
  PREFIX wbt: <http://diseases.semscape.org/prop/direct/>
  SELECT * WHERE {?item wbt:P4 ?doid}
"""
existing_do = dict()
results =  wdi_core.WDItemEngine.execute_sparql_query(query=query, endpoint="https://diseases.semscape.org/query/sparql")
pprint.pprint(results)
for result in results["results"]["bindings"]:
    existing_do[result["doid"]["value"]] = result["item"]["value"]

existing_do

In [ ]:
qres = doGraph.query(
    """
      

       SELECT ?source ?target WHERE {

       ?s <http://www.w3.org/2004/02/skos/core#exactMatch> ?o ;
          <http://www.w3.org/2002/07/owl#annotatedSource> ?source ;
          <http://www.w3.org/2002/07/owl#annotatedTarget> ?target .


  }
       """)

for row in qres:
  print(row[0], row[1])

In [ ]:
qres = doGraph.query(
    """
      SELECT DISTINCT ?source ?target ?doid ?mesh WHERE {
          ?source <http://www.w3.org/2004/02/skos/core#exactMatch> ?target .
          OPTIONAL {?source <http://www.w3.org/2004/02/skos/core#exactMatch> ?doid .}
          OPTIONAL {?source <http://www.w3.org/2004/02/skos/core#exactMatch> ?mesh .}
          FILTER REGEX(str(?doid), "DOID", "i")
      }
    """
)
for row in qres:
  print(row[0], row[1])

In [5]:
query = "SELECT * WHERE { ?qid wdt:P5270 ?mondoid}"
mondoinwd = wdi_core.WDFunctionsEngine.execute_sparql_query(query, as_dataframe=True)
mondoinwd
workdata = df_mondoNative.merge(mondoinwd, on="mondoid")
workdata

,mondo_uri,mondoid,label,exactMatches,aliases,qid
0,http://purl.obolibrary.org/obo/MONDO_0014972,MONDO:0014972,"chromosome 19q13.11 deletion syndrome, proximal",http://identifiers.org/omim/617219|http://link...,,http://www.wikidata.org/entity/Q55785152
1,http://purl.obolibrary.org/obo/MONDO_0017899,MONDO:0017899,autosomal dominant mendelian susceptibility to...,http://linkedlifedata.com/resource/umls/id/CN2...,autosomal dominant MSMD due to a partial defic...,http://www.wikidata.org/entity/Q55787487
2,http://purl.obolibrary.org/obo/MONDO_0019888,MONDO:0019888,distal trisomy 20q,http://identifiers.org/snomedct/764500002|http...,telomeric duplication 20q|distal duplication 2...,http://www.wikidata.org/entity/Q55788957
3,http://purl.obolibrary.org/obo/MONDO_0009339,MONDO:0009339,congenital bile acid synthesis defect 2,http://identifiers.org/mesh/C535443|http://ide...,"CBAS2|bile acid synthesis defect, congenital, ...",http://www.wikidata.org/entity/Q42863601
4,http://purl.obolibrary.org/obo/MONDO_0009583,MONDO:0009583,blepharophimosis - intellectual disability syn...,http://www.orpha.net/ORDO/Orphanet_2728|http:/...,"Ohdo-Madokoro-Sonoda syndrome|BMRS, Ohdo type|...",http://www.wikidata.org/entity/Q3961681
...,...,...,...,...,...,...
17687,http://purl.obolibrary.org/obo/MONDO_0033282,MONDO:0033282,multiple mitochondrial dysfunctions syndrome 5,http://purl.obolibrary.org/obo/DOID_0080274|ht...,,http://www.wikidata.org/entity/Q53661614
17688,http://purl.obolibrary.org/obo/MONDO_0024642,MONDO:0024642,gastric neuroendocrine tumor G2,http://linkedlifedata.com/resource/umls/id/C32...,gastric NET G2|gastric neuroendocrine tumor G2...,http://www.wikidata.org/entity/Q55790735
17689,http://purl.obolibrary.org/obo/MONDO_0002144,MONDO:0002144,obsolete hyperuricemia (disease),http://purl.obolibrary.org/obo/DOID_1920|http:...,hyperuricemia|uricacidemia|blood urate raized|...,http://www.wikidata.org/entity/Q49970
17690,http://purl.obolibrary.org/obo/MONDO_0021680,MONDO:0021680,streptococcal infection,http://identifiers.org/snomedct/85769006|http:...,Streptococcus caused disease or disorder|Strep...,http://www.wikidata.org/entity/Q5849703


In [23]:
from datetime import datetime
import copy

login = wdi_login.WDLogin("Andrawaag", "k1T0BoAUHKTj")

def createMONDOReference(doid):
    statedin = wdi_core.WDItemID("Q27468140", prop_nr="P248", is_reference=True)
    retrieved = datetime.now()
    timeStringNow = retrieved.strftime("+%Y-%m-%dT00:00:00Z")
    refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True)
    doid = wdi_core.WDExternalID(doid, prop_nr="P5270", is_reference=True)
    return [statedin, refRetrieved, doid]

def remove_mondo_ref(qid):
    item = wdi_core.WDItemEngine(wd_item_id=qid)
    json = item.get_wd_json_representation()

    for key in json["claims"].keys():
        for claim in json["claims"][key]:
            print(claim["id"])
            for reference in claim["references"]:
                for snakP248 in reference["snaks"]["P248"]:
                    if snakP248["datavalue"]["value"]["id"] == "Q55345445":
                        wdi_core.WDItemEngine.delete_statement(statement_id=claim["id"], revision=item.lastrevid, login=login)

def updateMondo(row):
    statements = []
    mondo_id = row["mondoid"]
    mondo_reference = createMONDOReference(mondo_id)

    # Mondo ID
    statements.append(wdi_core.WDExternalID(row["mondoid"], prop_nr="P5270", references=[copy.deepcopy(mondo_reference)]))
    statements.append(wdi_core.WDUrl(row["mondo_uri"], prop_nr="P2888", references=[copy.deepcopy(mondo_reference)]))

    # subclasses 
    query="""PREFIX obo: <http://www.geneontology.org/formats/oboInOwl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

        SELECT ?subClassOf WHERE {"""

    query+= "<"+row["mondo_uri"]+"> rdfs:subClassOf ?subClassOf .  FILTER (REGEX(str(?subClassOf), 'http', 'i'))} "

    for rij in doGraph.query(query):
        print(rij[0])
        for i,r in workdata[workdata["mondo_uri"]==str(rij[0])].iterrows():
            statements.append(wdi_core.WDItemID(r["qid"].replace("http://www.wikidata.org/entity/", ""), prop_nr="P279", references=[copy.deepcopy(mondo_reference)]))
    
    # exact matches
    for skosExactMatch in list(set(row["exactMatches"].split("|"))):
        if skosExactMatch != "":
            statements.append(wdi_core.WDUrl(skosExactMatch, prop_nr="P2888", references=[copy.deepcopy(mondo_reference)]))
            # MeSH descriptor ID (P486)
            # http://identifiers.org/mesh/
            if "http://identifiers.org/mesh/" in skosExactMatch:
                mesh = skosExactMatch.replace("http://identifiers.org/mesh/", "")
                statements.append(wdi_core.WDExternalID(mesh, prop_nr="P486", references=[copy.deepcopy(mondo_reference)]))
                # UMLS CUI (P2892)
                # http://linkedlifedata.com/resource/umls/id/
            if "http://linkedlifedata.com/resource/umls/id/" in skosExactMatch:
                umls = skosExactMatch.replace("http://linkedlifedata.com/resource/umls/id/", "")
                statements.append(wdi_core.WDExternalID(umls, prop_nr="P2892", references=[copy.deepcopy(mondo_reference)]))

            # Orphanet ID (P1550)
            # http://www.orpha.net/ORDO/Orphanet_
            if "http://www.orpha.net/ORDO/Orphanet_" in skosExactMatch:
                ordo = skosExactMatch.replace("http://www.orpha.net/ORDO/Orphanet_", "")
                statements.append(wdi_core.WDExternalID(ordo, prop_nr="P1550", references=[copy.deepcopy(mondo_reference)]))

            # SNOMED CT identifier (P5806)
            # http://identifiers.org/snomedct/191347008
            if "http://identifiers.org/snomedct/" in skosExactMatch:
                snomed = skosExactMatch.replace("http://identifiers.org/snomedct/", "")
                statements.append(wdi_core.WDExternalID(snomed, prop_nr="P5806", references=[copy.deepcopy(mondo_reference)]))

            # NCI Thesaurus ID (P1748)
            # http://purl.obolibrary.org/obo/NCIT_
            if "http://purl.obolibrary.org/obo/NCIT_" in skosExactMatch:
                ncit = skosExactMatch.replace("http://purl.obolibrary.org/obo/NCIT_", "NCIT:")
                statements.append(wdi_core.WDString(ncit, prop_nr="P1748", references=[copy.deepcopy(mondo_reference)]))

            # OMIM (P492)
            # http://identifiers.org/omim/
            if "http://identifiers.org/omim/" in skosExactMatch:
                omim = skosExactMatch.replace("http://identifiers.org/omim/", "")
                statements.append(wdi_core.WDExternalID(omim, prop_nr="P492", references=[copy.deepcopy(mondo_reference)]))



    wb_mondo_item = wdi_core.WDItemEngine(wd_item_id=row["qid"].replace("http://www.wikidata.org/entity/",""), data=statements, keep_good_ref_statements=True)

    if wb_mondo_item.get_label() == "":
        wb_mondo_item.set_label(row["label"], lang="en")
    if wb_mondo_item.get_description() == "":
        wb_mondo_item.set_description("human disease", lang="en")
    if row["aliases"] != "":
        aliases = wb_mondo_item.get_aliases()
        for new_alias in row["aliases"].split("|"):
            aliases.append(new_alias)
        wb_mondo_item.set_aliases(list(set(aliases)), lang="en")

    #pprint.pprint(wb_mondo_item.get_wd_json_representation())
    return wb_mondo_item.write(login)

for index, row in workdata.iterrows():
    qid = row["qid"].replace("http://www.wikidata.org/entity/", "")
    if qid != "":
        item = wdi_core.WDItemEngine(wd_item_id=qid)
        data = [wdi_core.WDBaseDataType.delete_statement(prop_nr="P5270")]
        item = wdi_core.WDItemEngine(wd_item_id=qid, data=data)
        print(item.write(login))
        remove_mondo_ref(qid)
        print(updateMondo(row))
        


Q55785152
Q55785152$5E23CBBA-BEF0-45C7-A730-033FA5711598
Q55785152$57CF6EC2-68F6-4B7D-A349-8D00603981C9
Q55785152$80139B9D-A91C-42E8-8655-65788715DBA1
Q55785152$BFB36CF5-E9EB-4844-826B-EC299BB70688
Q55785152$3B33E106-E63F-44B5-9DB4-64685061307B
Q55785152$F0D9BF5F-4BFB-4108-960C-CD68E9DD26CE
http://purl.obolibrary.org/obo/MONDO_0000761
Q55785152
Q55787487
Q55787487$F558F27D-21C5-4385-8EE3-1B07E527F3D3
{'pageinfo': {'lastrevid': 1318784844}, 'success': 1, 'claims': ['Q55787487$F558F27D-21C5-4385-8EE3-1B07E527F3D3']}
Q55787487$3BCEDB25-0DC1-469A-B9C6-CD4BC70F6ED9
{'pageinfo': {'lastrevid': 1318784858}, 'success': 1, 'claims': ['Q55787487$3BCEDB25-0DC1-469A-B9C6-CD4BC70F6ED9']}
Q55787487$C608E0FC-92A1-4599-AF75-1EEBA9D2FC7F
http://purl.obolibrary.org/obo/MONDO_0020637
Q55787487
Q55788957
Q55788957$8F6B6429-EE90-44B3-B336-14F17E62E464
{'pageinfo': {'lastrevid': 1318784909}, 'success': 1, 'claims': ['Q55788957$8F6B6429-EE90-44B3-B336-14F17E62E464']}
Q55788957$F13F7CEE-6B01-47B0-8887-1490FCE6

KeyboardInterrupt: 

In [ ]:
def createMONDOReference(doid):
    statedin = wdi_core.WDItemID("Q27468140", prop_nr="P248", is_reference=True)
    retrieved = datetime.now()
    timeStringNow = retrieved.strftime("+%Y-%m-%dT00:00:00Z")
    refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True)
    doid = wdi_core.WDExternalID(doid, prop_nr="P5270", is_reference=True)
    return [statedin, refRetrieved, doid]


In [ ]:
wb_mondo_item = wdi_core.WDItemEngine(data=statements, keep_good_ref_statements=True,)

aliases = "proliferative vasculopathy and hydranencephaly/hydrocephaly|Encephaloclastic proliferative vasculopathy|hydrocephaly/hydranencephaly due to cerebral vasculopathy|cerebral proliferative glomeruloid vasculopathy|proliferative vasculopathy and hydranencephaly/hydrocephaly|Encephaloclastic proliferative vasculopathy|hydrocephaly/hydranencephaly due to cerebral vasculopathy|cerebral proliferative glomeruloid vasculopathy|proliferative vasculopathy and hydranencephaly/hydrocephaly|Encephaloclastic proliferative vasculopathy|hydrocephaly/hydranencephaly due to cerebral vasculopathy|cerebral proliferative glomeruloid vasculopathy|proliferative vasculopathy and hydranencephaly/hydrocephaly|Encephaloclastic proliferative vasculopathy|hydrocephaly/hydranencephaly due to cerebral vasculopathy|cerebral proliferative glomeruloid vasculopathy|proliferative vasculopathy and hydranencephaly/hydrocephaly|Encephaloclastic proliferative vasculopathy|hydrocephaly/hydranencephaly due to cerebral vasculopathy|cerebral proliferative glomeruloid vasculopathy|proliferative vasculopathy and hydranencephaly/hydrocephaly|Encephaloclastic proliferative vasculopathy|hydrocephaly/hydranencephaly due to cerebral vasculopathy|cerebral proliferative glomeruloid vasculopathy"
if wb_mondo_item.get_label() == "":
    wb_mondo_item.set_label(row["label"], lang="en")
if wb_mondo_item.get_description() == "":
    wb_mondo_item.set_description("human disease", lang="en")
if row["aliases"] != "":
    aliases = wb_mondo_item.get_aliases()
    for new_alias in aliases.split("|"):
        aliases.append(new_alias)
    wb_mondo_item.set_aliases(aliases, lang="en")

pprint.pprint(wb_mondo_item.get_wd_json_representation())

In [21]:
query="""PREFIX obo: <http://www.geneontology.org/formats/oboInOwl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

        SELECT ?subClassOf WHERE {"""

query+= "<"+row["mondo_uri"]+"> rdfs:subClassOf ?subClassOf .  FILTER (REGEX(str(?subClassOf), 'http', 'i'))} "

for rij in doGraph.query(query):
    print(rij[0])
    for i,r in workdata[workdata["mondo_uri"]==str(rij[0])].iterrows():
        print(r["qid"].replace("http://www.wikidata.org/entity/", ""))

    
#if row[0].replace("http://purl.obolibrary.org/obo/DOID_", "DOID:") not in doQids.keys():
#    doQids[row[0].replace("http://purl.obolibrary.org/obo/DOID_", "DOID:")] = create(row[0].replace("http://purl.obolibrary.org/obo/DOID_", "DOID:"))
#    #statements.append(wdi_core.WDItemID(value=doQids[row[0].replace("http://purl.obolibrary.org/obo/DOID_", "DOID:")].replace("http://www.wikidata.org/entity/", ""),row["mondo_uri"],prop_nr="P279", references=[copy.deepcopy(do_reference)]))
#query="""PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

http://purl.obolibrary.org/obo/MONDO_0000761
Q16918398
